In [1]:
from magpie import grasp as gt
import rtde_control
import rtde_receive
from magpie.motor_code import Motors
from magpie import ur5 as ur5
import time
import numpy as np
import copy
import magpie
from magpie.gripper import Gripper

In [2]:
import copy
from PIL import Image
import open3d as o3d
import matplotlib.pyplot as plt
from magpie.perception import pcd
from open3d.web_visualizer import draw

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [3]:
from magpie import realsense_wrapper as real
rsc = real.RealSense()
rsc.initConnection()


In [4]:
from magpie.perception.label_owlvit import LabelOWLViT
path = "google/owlvit-base-patch32"
label_vit = LabelOWLViT(pth=path)

2024-07-12 12:29:47.733152: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 12:29:47.733178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 12:29:47.734033: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 12:29:47.739217: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 12:29:48.532971: W tensorflow/compiler/tf2

In [5]:
import importlib
importlib.reload(gt)
importlib.reload(ur5)
importlib.reload(magpie.gripper)

<module 'magpie.gripper' from '/home/will/MAGPIE/magpie/gripper.py'>

In [5]:
def seg(index):
    p, rgbd_image = rsc.getPCD()
    image = np.array(rgbd_image.color)
    queries = ["a photo of a yellow rubber duck"]
    # queries = ["a photo of a tail"]
    # queries = ["a photo of a black handle of a pair of scissors"]
    abbrevq = ["duck"]
    label_vit.set_threshold(0.005)
    # bboxes, uboxes = label_vit.label(image, queries, abbrevq, topk=True, plot=True)
    bboxes, uboxes = label_vit.label(image, queries, abbrevq, topk=True, plot=False)
    rgbd_image, cpcd, tmat, pcaFrame = pcd.get_segment(
                                            label_vit.sorted_labeled_boxes_coords, 
                                            # label_vit.boxes, 
                                            index, 
                                            rgbd_image, 
                                            rsc, 
                                            type="box-dbscan", 
                                            #  type="box", 
                                            #  method="quat", 
                                            method="iterative", 
                                            #  display=False,
                                            display=False,
                                            viz_scale=1000)
    tmat, tmat[:3, 3]
    
    return cpcd, tmat

In [6]:
# load arrays in saved_poses/ folder as array of arrays
saved_poses = []
for i in range(5):
    saved_poses.append(np.load(f'saved_poses/pose_{i+1}.npy', allow_pickle=True))

In [ ]:
print(len(saved_poses))

In [ ]:
tmats = []

In [8]:
ur = ur5.UR5_Interface()
try:
    ur.start()
    currentPose = ur.getPose()
    ur.open_gripper()
    ur.stop()

except Exception as e:
    ur.stop()
    raise e


AttributeError: 'NoneType' object has no attribute 'servoStop'

In [ ]:
ur = ur5.UR5_Interface()
try:
    ur.start()
    currentPose = ur.getPose()
    ur.close_gripper()
    ur.stop()

except Exception as e:
    ur.stop()
    raise e

In [ ]:
ur = ur5.UR5_Interface()
sleepRate = 3

try:
    ur.start()
    currentPose = ur.getPose()
    ur.moveL(saved_poses[1])
    time.sleep(sleepRate)
    ur.open_gripper()
    cpcd, tmat = seg(index=0)
    
    tmat_gripper = np.array([[1, 0, 0, 0],
                                 [0, 1, 0, 0],
                                 [0, 0, 1, 0.08],
                                 [0, 0, 0, 1]])

    ee_to_base = ur.get_tcp_pose()

    full_transform = ee_to_base 

    o3d.visualization.draw_geometries([cpcd])
    
    print(full_transform)
    #ur.moveL(full_transform)

    time.sleep(sleepRate)



    ur.stop()

except Exception as e:
    ur.stop()
    raise e

In [ ]:
ur = ur5.UR5_Interface()
sleepRate = 3

try:
    ur.start()
    currentPose = ur.get_tcp_pose()
    print(currentPose)
except Exception as e:
    ur.stop()
    raise e

In [ ]:
ur = ur5.UR5_Interface()
try:
    ur.start()
    np.save('saved_poses_2/pose_down.npy', ur.get_tcp_pose())
except Exception as e:
    ur.stop()
    raise e

In [ ]:
ur = ur5.UR5_Interface()
sleepRate = 3

try:
    ur.start()
    currentPose = ur.getPose()
    p = np.load(f'saved_poses_2/pose_down.npy', allow_pickle=True)
    ur.moveL(p)
    time.sleep(sleepRate)


    ur.stop()

except Exception as e:
    ur.stop()
    raise e

In [11]:
name = "pose_5"
sleepRate = 3

ur = ur5.UR5_Interface()
try:
    ur.start()
    currentPose = ur.getPose()
    print(currentPose)

    merged_pcd = o3d.geometry.PointCloud()
    
    for k, target_pose in enumerate(saved_poses):
        ur.moveL(target_pose)
        time.sleep(sleepRate)

        # Capture point cloud and transformation matrix
        cpcd, tmat = seg(index=1 if k == 4 else 0)

        rotation_matrix = np.array([[0, 1, 0, 0],
                            [-1, 0, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])

        #tmat = tmat @ rotation_matrix

        o3d.io.write_point_cloud(f"saved_pcd/cpcd{k+1}.ply", cpcd)

        # Transformation matrix for gripper length
        tmat_gripper = np.array([[1, 0, 0, 0],
                                 [0, 1, 0, 0],
                                 [0, 0, 1, 0.08],
                                 [0, 0, 0, 1]])

        # Get the transformation matrix of the end effector in the base frame
        ee_to_base = ur.get_tcp_pose()

        
        cpcd.transform(rotation_matrix)  # End effector to base frame
        cpcd.transform(ee_to_base)  # End effector to base frame

        # Save the transformed point cloud
        o3d.io.write_point_cloud(f"saved_pcd/cpcd{k+1}_t.ply", cpcd)

        # Use ICP to align the new point cloud with the merged point cloud
        if k == 0:
            merged_pcd = cpcd
        else:
            threshold = 0.055  # Distance threshold for ICP
            init_transform = np.eye(4)  # Initial guess for the transformation
            reg_p2p = o3d.pipelines.registration.registration_icp(
                cpcd, merged_pcd, threshold, init_transform,
                o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1800))
            
            # Transform the new point cloud using the result of ICP
            cpcd.transform(reg_p2p.transformation)
            
            # Merge the point clouds
            merged_pcd += cpcd

    ur.stop()
except Exception as e:
    ur.stop()
    raise e

# Save the merged point cloud
o3d.io.write_point_cloud(f"saved_pcd/merged_pcd.ply", merged_pcd)

# Visualize the merged point cloud
o3d.visualization.draw_geometries([merged_pcd])

AttributeError: 'NoneType' object has no attribute 'servoStop'

In [ ]:
ur.stop()

In [ ]:
# Capture point cloud and transformation matrix
cpcd, tmat = seg(index=0)
o3d.io.write_point_cloud(f"saved_pcd/cpcd{1}.ply", cpcd)

# Transformation matrix for gripper length
tmat_gripper = np.array([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])

# Get the transformation matrix of the end effector in the base frame
ee_to_base = ur.get_tcp_pose()


cpcd.transform(ee_to_base)  # End effector to base frame

# Save the transformed point cloud
o3d.io.write_point_cloud(f"saved_pcd/cpcd{1}_t.ply", cpcd)


In [ ]:
merged = o3d.io.read_point_cloud(f"saved_pcd/cpcd1_t.ply")
o3d.visualization.draw_geometries([merged])

In [ ]:
# set matrix
# tmat = np.array([[ 0.91574736, -0.35602909,  0.18614527, -0.02848649],
#        [-0.40073104, -0.77640133,  0.48643151,  0.14332736],
#        [ 0.02866033,  0.52004256,  0.85365937,  0.33750796],
#        [ 0.        ,  0.        ,  0.        ,  1.        ]])

tmat = np.array([[ 0.98665647, -0.16180182, -0.01814324,  0.04438557],
       [-0.1565328 , -0.97333117,  0.16770185,  0.04194964],
       [ 0.04479385,  0.1626241 ,  0.98567079,  0.32025074],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

aperture = 37
# z_offset = G.aperture_to_z(aperture)/1000.0

mmc = copy.deepcopy(tmat[:3, 3])
# grasp_pose = [mmc[1], -mmc[0], mmc[2]]
grasp_pose = mmc
tmat[:3, 3] = [0, 0, 0]
homePose = None
z_offset = 100/1000.0
z_offset -= 0.035
z_offset

In [ ]:
# move gripper to cartesian pos
# OR orient gripper in place
name = "pose_5"
sleepRate = 4
# time.sleep(sleepRate)
ur = ur5.UR5_Interface()
try:
    ur.start()
    currentPose = ur.getPose()
    print(currentPose)

    ur.moveL(saved_poses[4])
    time.sleep(sleepRate)
    time.sleep(sleepRate)
    ur.stop()
except Exception as e:
    ur.stop()
    raise(e)

In [ ]:
index = 1
# this does the [x, y, z] --> [y, -x, z] grasp pose switch, and the -y inversio on the y-axis orientation
# rgbd_image, cpcd, tmat = pcd|.get_segment(label_vit.boxes, index, rgbd_image, rsc, type="box", display=False)
rgbd_image, cpcd, tmat, pcaFrame = pcd.get_segment(
                                        label_vit.sorted_labeled_boxes_coords, 
                                        # label_vit.boxes, 
                                         index, 
                                         rgbd_image, 
                                         rsc, 
                                         type="box-dbscan", 
                                        #  type="box", 
                                        #  method="quat", 
                                         method="iterative", 
                                        #  display=False,
                                         display=True,
                                         viz_scale=1000)
tmat, tmat[:3, 3]



In [ ]:
# move gripper to cartesian pos
# OR orient gripper in place
sleepRate = 1.5
# time.sleep(sleepRate)
ur = ur5.UR5_Interface()
try:
    ur.start()
    # homePose = ur.get_tcp_pose()
    homePose = ur.getPose()
    currentPose = ur.getPose()
    desiredPose = np.matmul(np.array(currentPose), tmat)
    print("Desired Pose: ", desiredPose)
    # in-place (or best attempt) re-orientation
    # ur.moveL(desiredPose)
    ## move to cartesian pos
    gt.move_to_L(grasp_pose, ur, z_offset=z_offset)    
    print("Done moving to block")
    time.sleep(sleepRate)
    time.sleep(sleepRate)
    ur.stop()
except Exception as e:
    ur.stop()
    raise(e)

In [ ]:
# move home
ur = ur5.UR5_Interface()
try:
    ur.start()
    # time.sleep(sleepRate)
    # homePose = np.array([[-0.024, -0.998, -0.062, -0.261],
    #                     [-0.999,  0.021,  0.035, -0.162],
    #                     [-0.033,  0.063, -0.997,  0.221],
    #                     [ 0.   ,  0.   ,  0.   ,  1.   ]])
    ur.moveL(homePose)
    # gt_home = ur.getPose()
    # print(np.array(gt_home))
    time.sleep(sleepRate * 3)
    ur.stop()
except Exception as e:
    ur.stop()

In [ ]:
# DO NOT EXECUTE

In [ ]:
# # orient and move gripper to cartesian pos
# sleepRate = 1.5
# ur = ur5.UR5_Interface()
# try:
#     servoPort = "/dev/ttyACM0"
#     ur.start()
#     time.sleep(sleepRate)
#     homePose = ur.get_tcp_pose()    
#     '''
#     stopping point 2/15:
#     this works so far
#     take home pos
#     add translation on the gripper frame to the goal x, y
#     apply tmat to the goal
#     move to goal (translates on x, y and orients at the same time)
#     ## does this address the fact that orientation is not achievable in standstill?

#     then: take orig Z (from original gripper frame): [0, 0, dZ]
#     transform by tmat (the current orientation of the gripper)
#     add transformed dZ to the goal position
#     move
#     works!
#     '''
#     dX,dY,dZ = gt.get_world_frame(grasp_pose, ur, z_offset=z_offset)

#     goal1 = copy.deepcopy(homePose)
#     goal1 = np.array(goal1)
#     # goal1[:3, 3] += [dX, dY, 0]
#     goal1[:3, 3] += [dX, dY, dZ]
#     goal1 = np.array(goal1) @ tmat
#     ur.moveL(goal1)
#     time.sleep(sleepRate)

#     # todo: take currentPose and validate orientation?
#     goal2 = goal1
#     posd = [0, 0, dZ] @ tmat[:3, :3]
#     goal2 = np.array(goal2)
#     goal2[:3, 3] += posd
#     # ur.moveL(goal2)
#     time.sleep(sleepRate)
#     '''
#     end of working section
#     '''
#     ur.stop()
# except Exception as e:
#     ur.stop()
#     raise(e)